# Creating a synthetic Q&A dataset

In [1]:
import os

proxy = 'http://dell-1.star:7890' # 3090 docker
# proxy = 'http://127.0.0.1:7890' # clash
# proxy = 'http://127.0.0.1:1080' # naiveproxy

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

## Read in data, and create a context

In [2]:
import pandas as pd
df = pd.read_csv("docx/qa/total.csv")
df['context'] = "summary: " +  df.summary + "\n" +  "content: " + df.content
df.head()

,summary,content,summary_token_num,content_token_num,token_num,context
0,电信业务#电信业务分类目录中的基础电信业务有哪些,答案要点：(一)固定网络国内长途及本地电话业务；(二)移动网络电话和数据业务；(三)卫星通信...,13,104,117,summary: 电信业务#电信业务分类目录中的基础电信业务有哪些\ncontent: 答案...
1,电信业务#电信业务分类目录中的增值电信业务有哪些,答案要点：(一)电子邮件；(二)语音信箱；（三）在线信息库存储和检索；(四)电子数据交换；(...,13,66,79,summary: 电信业务#电信业务分类目录中的增值电信业务有哪些\ncontent: 答案...
2,电信业务#怎样连上Internet,答案要点：一般说来，连入Internet主要有异步拨号和专线接入这两种方式。异步方式是指通过...,8,118,126,summary: 电信业务#怎样连上Internet\ncontent: 答案要点：一般说来...
3,电信业务#何为ＥＣ（电子商务,答案要点：究竟什么是电子商务，不同的人往往有不同的理解。电子商务最通常的一个定义是：交易所涉...,8,74,82,summary: 电信业务#何为ＥＣ（电子商务\ncontent: 答案要点：究竟什么是电子...
4,电信业务#如何在网络中共享打印机,"答案要点：单击""开始""，指向""设置""，然后单击""打印机""。单击要共享的打印机。在""文件""菜单...",9,45,54,"summary: 电信业务#如何在网络中共享打印机\ncontent: 答案要点：单击""开始..."


## Create questions based on the context

In [3]:
import openai
openai.api_key_path = ".openai-key"

In [5]:
import requests

proxy = 'http://dell-1.star:7890'
try:
    resp = requests.get("http://google.com", timeout=5, proxies={"http": proxy, "https": proxy})
    assert resp.status_code == 200
except Exception as e:
    print(e)
# print(resp.status_code)

context = """
summary: 网速慢的处理方法#检查是否硬盘空间不够，删除一些临时文件#MODE旁边有高功率的电器设备，很容易导致信号受干扰#建议用户在自己的计算机上安装杀毒软件，及时发现和查杀病毒•
content: 第八章建立连接正常但无法收发E-mail此情况一般是电脑设置问题。您可先了解一下，如果是其他ADSL用户可发，您不能发，请检查您的电脑如检查您是否设置了SmtP认证、发送邮件（SMTP服务器等如个别用户只可发小文件，不能发大文件，是因大多数E-mail服务器对附件的大小有限制。因此，在软件上应对较大的附件进行分拆设置，具体操作，可见E-mail软件的相关帮助
"""

In [27]:

response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=f"请根据以下文本生成问题，尽可能使用简体中文，问题表述需要清晰详细\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
print(response.choices[0].text)

questions = "1." + response.choices[0].text
resp = openai.Completion.create(
            engine="text-davinci-003",
            prompt=f"根据以下文本生成问题的答案，尽可能使用简体中文\n\n文本: {context}\n\n问题:\n{questions}\n\n答案:\n1.",
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
print(resp['choices'][0]['text'])

 如何检查硬盘空间是否不够？
2. 为什么MODE旁边有高功率的电器设备会导致信号受干扰？
3. 如何在自己的计算机上安装杀毒软件？
4. 如何检查电脑是否设置了SmtP认证、发送邮件（SMTP服务器）？
5. 为什么有些用户只可发小文件，不能发大文件？
6. 如何对较大的附件进行分拆设置？
 可以使用计算机的磁盘管理工具查看硬盘空间是否不够。
2. 高功率的电器设备会产生电磁干扰，影响MODE的信号传输。
3. 可以下载安装杀毒软件，然后根据提示进行安装。
4. 可以在计算机的网络设置中查看是否设置了SmtP认证、发送邮件（SMTP服务器）。
5. 大多数E-mail服务器对附件的大小有限制，因此有些用户只可发小文件，不能发大文件。
6. 可以在E-mail软件的设置中，对较大的附件进行分拆设置。


In [10]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"请根据以下文本生成问题，尽可能使用简体中文，问题表述需要清晰详细\n\n文本: {context}\n\n问题:\n1."
        }
    ],
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response.choices[0].message.content)

questions = "1." + response.choices[0].message.content
resp = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"根据以下文本生成问题的答案，尽可能使用简体中文\n\n文本: {context}\n\n问题:\n{questions}\n\n答案:\n1."
        }
    ],
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(resp.choices[0].message.content)

网速慢的处理方法有哪些？
2. 为什么MODE旁边的电器设备会影响信号？
3. 建议用户在自己的计算机上安装什么软件？
4. 电脑设置问题可能导致什么情况？
5. 如何解决不能发送大文件的问题？
检查是否硬盘空间不够，删除一些临时文件；建议用户在自己的计算机上安装杀毒软件，及时发现和查杀病毒。
2. 高功率的电器设备容易导致信号受干扰。
3. 建议用户在自己的计算机上安装杀毒软件，及时发现和查杀病毒。
4. 电脑设置问题可能导致无法收发E-mail。
5. 对较大的附件进行分拆设置，具体操作可见E-mail软件的相关帮助。


In [7]:
# use davinci 
def get_questions(context):
    try:
        response = openai.Completion.create(
            # engine="davinci-instruct-beta-v3",
            engine="text-davinci-003",
            prompt=f"请根据以下文本生成问题,尽可能使用简体中文\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response['choices'][0]['text']
    except:
        return ""

df['questions']= df.context.apply(get_questions)
df['questions'] = "1." + df.questions

In [11]:
# use gpt-3.5-turbo
def get_questions_by_chat(context):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": f"请根据以下文本生成问题，尽可能使用简体中文，问题表述需要清晰详细\n\n文本: {context}\n\n问题:\n1."
                }
            ],
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response.choices[0].message.content
    except:
        return ""
df['questions']= df.context.apply(get_questions_by_chat)
df['questions'] = "1." + df.questions

In [12]:
print(df[['questions']].values)

[['1.电信业务分类目录中的基础电信业务有哪些？\n2. 固定网络国内长途及本地电话业务属于哪一类基础电信业务？\n3. 互联网及其它公共数据传送业务属于哪一类基础电信业务？\n4. 带宽、波长、光纤、光缆、管道及其它网络元素出租、出售业务属于哪一类基础电信业务？\n5. 无线寻呼业务属于哪一类基础电信业务？']
 ['1.电信业务分类目录中有哪些增值电信业务？\n2. 列举电信业务分类目录中的增值电信业务。\n3. 电子邮件、语音信箱、在线信息库存储和检索等是电信业务分类目录中的哪些增值电信业务？\n4. 电信业务分类目录中的增值传真、互联网接入服务、可视电话会议服务等有哪些？']
 ['1.连入Internet主要有哪两种方式？\n2. 异步方式和专线接入方式各有什么优缺点？\n3. 哪种方式适合普通用户？哪种方式适合将整个计算机局域网接入Internet？']
 ['1.电子商务的最通常定义是什么？\n2. 电子商务的本质特征是否能够被一个定义完全概括？\n3. 人们在实际生活中如何看待电子商务？\n4. 什么样的商业交易可以被视为电子商务？\n5. 电子商务是否需要通过物理交换或直接物理接触进行交易？']
 ['1.如何在网络中共享打印机？\n2. 具体步骤是什么，如何设置共享选项？\n3. 在哪个菜单中可以找到共享选项？\n4. 如何选择要共享的打印机？']
 ['1.如何查看自己的IP地址？\n2. 哪个程序可以帮助我们查看IP地址？\n3. 在哪个目录下可以找到WINIPCFG.EXE程序？']
 ['1.GSM是什么意思？ \n2. GSM采用什么技术？ \n3. GSM有哪些特点？ \n4. GSM能够为客户带来什么方便？ \n5. GSM能够开放哪些增值业务？']
 ['1.什么是SIM卡？\n2. SIM卡是由什么制成的？\n3. SIM卡的全称是什么？\n4. SIM卡有什么作用？']
 ['1.SIM卡的主要作用是什么？\n2. SIM卡具有哪些功能？\n3. GSM业务中，SIM卡的作用是什么？']
 ['1.SIM卡ICCID共有多少位？格式是怎样的？\n2. SIM卡ICCID中的各个数字和字母代表什么意思？\n3. SIM卡ICCID中的检验位是什么？有什么作用？\n4. SIM卡ICCID中的存储容量是以什么单位表示的？']
 

## Create answers based on the context

In [9]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            # engine="davinci-instruct-beta-v3",
            engine="text-davinci-003",
            prompt=f"根据以下文本生成问题的答案,尽可能使用简体中文\n\n文本: {row.context}\n\n问题:\n{row.questions}\n\n答案:\n1.",
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""

df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1." + df.answers

This model's maximum context length is 4097 tokens, however you requested 4671 tokens (4171 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 4097 tokens, however you requested 4773 tokens (4273 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 4097 tokens, however you requested 4931 tokens (4431 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.


In [13]:
def get_answers_by_chat(row):
    try:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": f"根据以下文本生成问题的答案，尽可能使用简体中文\n\n文本: {row.context}\n\n问题:\n{row.questions}\n\n答案:\n1."
                }
            ],
            temperature=0,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print (e)
        return ""

df['answers']= df.apply(get_answers_by_chat, axis=1)
df['answers'] = "1." + df.answers

The server is overloaded or not ready yet.
The server is overloaded or not ready yet.
The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID de09d81461cefc5f8b56a7110bf1a2ed in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID de09d81461cefc5f8b56a7110bf1a2ed in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID de09d81461cefc5f8b56a7110bf1a2ed in your mess

In [14]:
print(df[['answers']].values)

[['1.基础电信业务包括固定网络国内长途及本地电话业务、移动网络电话和数据业务、卫星通信及卫星移动通信业务、互联网及其它公共数据传送业务、带宽、波长、光纤、光缆、管道及其它网络元素出租、出售业务、网络承载、接入及网络外包等业务、国际通信基础设施、国际电信业务、无线寻呼业务和转售的基础电信业务。\n2. 固定网络国内长途及本地电话业务属于基础电信业务中的一类。\n3. 互联网及其它公共数据传送业务属于基础电信业务中的一类。\n4. 带宽、波长、光纤、光缆、管道及其它网络元素出租、出售业务属于基础电信业务中的一类。\n5. 无线寻呼业务属于基础电信业务中的一类。']
 ['1.电信业务分类目录中有增值电信业务。\n2. 增值电信业务包括电子邮件、语音信箱、在线信息库存储和检索、电子数据交换、在线数据处理与交易处理、增值传真、互联网接入服务、互联网信息服务、可视电话会议服务。\n3. 电子邮件、语音信箱、在线信息库存储和检索是电信业务分类目录中的增值电信业务。\n4. 增值传真、互联网接入服务、可视电话会议服务是电信业务分类目录中的增值电信业务。']
 ['1.连入Internet主要有异步拨号和专线接入这两种方式。\n2. 异步方式投资小，配置简单，但速度较慢，易受干扰，适于普通用户。专线接入方式投资大，但是比较稳定，速度快，适于将整个计算机局域网接入Internet。\n3. 异步方式适合普通用户，专线接入方式适合将整个计算机局域网接入Internet。']
 ['1.电子商务最通常的一个定义是：交易所涉及的各方通过电子手段而不是通过物理交换或直接物理接触而进行的任何形式的商业交易。\n2. 不能完全概括电子商务的本质特征。\n3. 人们常常将一切使用新的通信技术和计算机技术来改变传统的商务服务模式活动的都视为电子商务。\n4. 交易所涉及的各方通过电子手段而不是通过物理交换或直接物理接触而进行的任何形式的商业交易可以被视为电子商务。\n5. 不需要通过物理交换或直接物理接触进行交易。']
 ['1.单击"开始"，指向"设置"，然后单击"打印机"。单击要共享的打印机。在"文件"菜单上，单击"属性"。单击"共享"选项卡，然后单击"共享为"。\n2. 具体步骤是：单击"开始"，指向"设置"，然后单击"打印机"。单击要共享的打印机。在"文件"菜单上，单击"属性"。单击"共

In [15]:
df = df.dropna().reset_index().drop('index',axis=1)
df.to_csv("docx/qa/qa-chinese-gpt.csv", index=False)

## Debug proxy 

In [16]:
test_context = df.context[0]
response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题\n\n文本: {test_context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=800,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )


In [17]:
print(response['choices'][0]['text'])

 中国联通采用的是什么标准?
2. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包括可视   、无线上网，手机上网、手机电视、手机音乐等多种信息服务。

